In [1]:
pip install shap lime


     ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
     ----- ---------------------------------- 41.0/275.7 kB ? eta -:--:--
     ----- ---------------------------------- 41.0/275.7 kB ? eta -:--:--
     -------- ---------------------------- 61.4/275.7 kB 550.5 kB/s eta 0:00:01
     -------- ---------------------------- 61.4/275.7 kB 550.5 kB/s eta 0:00:01
     -------- ---------------------------- 61.4/275.7 kB 550.5 kB/s eta 0:00:01
     -------- ---------------------------- 61.4/275.7 kB 550.5 kB/s eta 0:00:01
     -------- ---------------------------- 61.4/275.7 kB 550.5 kB/s eta 0:00:01
     ---------- -------------------------- 81.9/275.7 kB 229.7 kB/s eta 0:00:01
     ---------- -------------------------- 81.9/275.7 kB 229.7 kB/s eta 0:00:01


In [1]:
import pandas as pd
import shap
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

fraud_data = pd.read_csv('C:/Users/nejat/AIM Projects/week8 data/Fraud_Data_cleaned.csv')

X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import shap

if 'purchase_time' in X_fraud.columns:
    X_fraud['purchase_time'] = pd.to_datetime(X_fraud['purchase_time'], errors='coerce')
    X_fraud['year'] = X_fraud['purchase_time'].dt.year
    X_fraud['month'] = X_fraud['purchase_time'].dt.month
    X_fraud['day'] = X_fraud['purchase_time'].dt.day
    X_fraud.drop('purchase_time', axis=1, inplace=True)

for col in X_fraud.select_dtypes(include=['int64']).columns:
    X_fraud[col] = X_fraud[col].astype('int32')


threshold = 100  
for col in ['device_id', 'source', 'browser', 'sex']:
    if col in X_fraud.columns:
        value_counts = X_fraud[col].value_counts()
        rare_categories = value_counts[value_counts < threshold].index
        X_fraud[col] = X_fraud[col].replace(rare_categories, 'Other')

categorical_cols = ['device_id', 'source', 'browser', 'sex']

X_fraud = pd.get_dummies(X_fraud, columns=categorical_cols, drop_first=True, sparse=True)

print("NaN values in X_fraud before filling:")
print(X_fraud.isnull().sum())

X_fraud.fillna(0, inplace=True)

y_fraud = y_fraud[X_fraud.index]  

rf_model_fraud = RandomForestClassifier(n_estimators=100)
rf_model_fraud.fit(X_fraud.toarray(), y_fraud)  

print("Model training completed successfully.")


explainer_fraud = shap.TreeExplainer(rf_model_fraud)
shap_values_fraud = explainer_fraud.shap_values(X_fraud.toarray())

shap.summary_plot(shap_values_fraud[1], X_fraud.toarray())


In [6]:
print(X_fraud.columns)


Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'age', 'ip_address', 'transaction_count', 'hour_of_day',
       'day_of_week', 'source_Direct', 'source_SEO', 'browser_FireFox',
       'browser_IE', 'browser_Opera', 'browser_Safari', 'sex_M'],
      dtype='object')
